## Conexion de Colab au drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importation des Packages

In [ ]:
# Importation des Packages

import modin.pandas as pd
import numpy as np
from tqdm import tqdm  # Affichage de barres de progression
import os


In [ ]:
!pip install modin[ray] openpyxl
import ray
ray.shutdown()  # redémarre proprement Ray s’il était déjà actif
ray.init(ignore_reinit_error=True)

2025-04-20 17:14:39,456	INFO worker.py:1852 -- Started a local Ray instance.


Python version:,3.11.12
Ray version:,2.44.1


In [ ]:
file_path = "/content/drive/MyDrive/Memoire_Bac/data_csv"
file = os.listdir(file_path)
# Affichage
for f in file:
    print(f)

2006.csv
2007.csv
2009.csv
2011.csv
2013.csv
2014.csv
2018.csv
2019.csv
2021.csv
2010.csv
2024.csv
2008.csv
2022.csv
2012.csv
2017.csv
2020.csv
2016.csv
2023.csv
2015.csv


In [ ]:
# Liste des années
annees = [str(a) for a in range(2006, 2024+1)]
data = {}

dtype_dict = {
    "resultat": "string",
    "mention": "string",
    "serie": "string",
    # "moy_finale": "float",
    "date_nais": "string"
    # ajoute les colonnes selon ton fichier
}
# Pour stocker les résultats
success_files = []
failed_files = []

# Chargement avec barre de progression
pbar = tqdm(annees, desc="Importation des fichiers CSV")
for annee in pbar:
    fichier = f"{file_path}/{annee}.csv"
    try:
        df = pd.read_csv(fichier, sep=";", encoding="utf-8", dtype=dtype_dict)  # Charge chaque fichier
        data[annee] = df  # Stocke dans le dictionnaire
        success_files.append(annee)
        pbar.set_postfix_str(f"{annee}.csv chargé ✅")
    except Exception as e:
        failed_files.append((annee, str(e)))
        print(f"❌ Erreur inconnue avec {fichier} : {e}")

if success_files:
    print(f"\n✅ Fichiers chargés avec succès ({len(success_files)}):")
    print(", ".join(success_files))

if failed_files:
    print(f"\n❌ Fichiers en erreur ({len(failed_files)}):")
    for annee, err in failed_files:
        print(f"  - {annee}.csv : {err}")

Importation des fichiers CSV:  79%|███████▉  | 15/19 [00:26<00:06,  1.60s/it, 2020.csv chargé ✅]UserWarning: `read_*` implementation has mismatches with pandas:
Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.
Importation des fichiers CSV: 100%|██████████| 19/19 [00:29<00:00,  1.56s/it, 2024.csv chargé ✅]


✅ Fichiers chargés avec succès (19):
2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024


In [ ]:
# Vérification des colonnes
colonnes_reference = set(data["2006"].columns)  # Colonnes de 2006 comme référence

for annee, df in data.items():
    colonnes_actuelles = set(df.columns)
    if colonnes_actuelles != colonnes_reference:
        print(f"⚠️ Les colonnes de {annee}.csv sont différentes de celles de 2006.csv")
        print(f"Colonnes manquantes dans {annee}.csv :", colonnes_reference - colonnes_actuelles)
        print(f"Colonnes supplémentaires dans {annee}.csv :", colonnes_actuelles - colonnes_reference)
    else:
        print(f"✅ Les colonnes de {annee}.csv sont identiques à celles de 2006.csv")


✅ Les colonnes de 2006.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2007.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2008.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2009.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2010.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2011.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2012.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2013.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2014.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2015.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2016.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2017.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2018.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2019.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2020.csv sont identiques à celles de 2006.csv
✅ Les colonnes de 2021.csv sont identiqu

In [ ]:
# prompt: afiche moi les colonnes dans data["2006"]

print(data["2006"].columns)


Index(['prenom', 'nom', 'date_nais', 'nationalite', 'serie', 'sexe', 'age',
       'etablissement', 'centre_examen', 'm_o_1', 'centre_ecrit',
       'type_candidat', 'resultat', 'acad_provenance', 'acad_ecrit',
       'moy_finale', 'mention', 'abs', 'exclusion'],
      dtype='object')


## Netoyage de 2006

In [ ]:
data["2006"].info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 42095 entries, 0 to 42094
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prenom           42095 non-null  object
 1   nom              42095 non-null  object
 2   date_nais        42095 non-null  string
 3   nationalite      42095 non-null  object
 4   serie            42095 non-null  string
 5   sexe             42095 non-null  object
 6   age              42095 non-null  int64 
 7   etablissement    42095 non-null  object
 8   centre_examen    42095 non-null  object
 9   m_o_1            28140 non-null  object
 10  centre_ecrit     41661 non-null  object
 11  type_candidat    42095 non-null  object
 12  resultat         40992 non-null  string
 13  acad_provenance  42095 non-null  object
 14  acad_ecrit       42095 non-null  object
 15  moy_finale       42095 non-null  object
 16  mention          24350 non-null  string
 17  abs              42095 non

In [ ]:
# prompt: je veux que tu me creer une colonne nommer annnee pour pour chaque fichier il prend comme valeur l'annee de sa clé

# Ajout de la colonne 'annee' à chaque DataFrame
for annee, df in data.items():
    df['annee'] = int(annee)  # Convertit l'année en entier
data["2006"].info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 42095 entries, 0 to 42094
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prenom           42095 non-null  object
 1   nom              42095 non-null  object
 2   date_nais        42095 non-null  string
 3   nationalite      42095 non-null  object
 4   serie            42095 non-null  string
 5   sexe             42095 non-null  object
 6   age              42095 non-null  int64 
 7   etablissement    42095 non-null  object
 8   centre_examen    42095 non-null  object
 9   m_o_1            28140 non-null  object
 10  centre_ecrit     41661 non-null  object
 11  type_candidat    42095 non-null  object
 12  resultat         40992 non-null  string
 13  acad_provenance  42095 non-null  object
 14  acad_ecrit       42095 non-null  object
 15  moy_finale       42095 non-null  object
 16  mention          24350 non-null  string
 17  abs              42095 non

In [ ]:
# prompt: je veux maintenat que tu me regroupe tout c'est données sur une même table les unes sur les autres

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data.values(), ignore_index=True)

# Display the first few rows of the combined DataFrame
all_data.head()


,prenom,nom,date_nais,nationalite,serie,sexe,age,etablissement,centre_examen,m_o_1,centre_ecrit,type_candidat,resultat,acad_provenance,acad_ecrit,moy_finale,mention,abs,exclusion,annee
0,Ndèye Bintou,DIOUF,31/12/1983,SENEGAL,L'1,F,22,CANDIDAT INDIVIDUEL DAKAR (OB),DAKAR,Anglais,LYCEE LAMINE GUEYE,Indiv./Libre,000,DK,DK,"05,42",<NA>,0,0,2006
1,Maïmouna,NDIAYE,19/02/1982,SENEGAL,L'1,F,24,CANDIDAT INDIVIDUEL DAKAR (OB),DAKAR,Anglais,LYCEE LAMINE GUEYE,Indiv./Libre,101,DK,DK,"10,25",PASS,0,0,2006
2,Maïmouna,MBAYE,14/11/1986,SENEGAL,L'1,F,19,CANDIDAT INDIVIDUEL DAKAR (OB),DAKAR,Anglais,LYCEE LAMINE GUEYE,Indiv./Libre,111,DK,DK,"11,17",PASS,0,0,2006
3,Fatimata,DIA,05/04/1984,SENEGAL,L'1,F,22,CANDIDAT INDIVIDUEL DAKAR (OB),DAKAR,Anglais,INSTITUTION NOTRE DAME,Indiv./Libre,100,DK,DK,"09,00",<NA>,0,0,2006
4,Fanny Mellysande,NGUIAS,26/08/1979,GABON,L'1,F,26,CANDIDAT INDIVIDUEL DAKAR (OB),DAKAR,Espagnol,INSTITUTION SAINT JEANNE D'ARC,Indiv./Libre,000,DK,DK,"05,83",AJ2,0,0,2006


In [ ]:
all_data.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 2236489 entries, 0 to 2236488
Data columns (total 20 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   prenom           object
 1   nom              object
 2   date_nais        string
 3   nationalite      object
 4   serie            string
 5   sexe             object
 6   age              object
 7   etablissement    object
 8   centre_examen    object
 9   m_o_1            object
 10  centre_ecrit     object
 11  type_candidat    object
 12  resultat         string
 13  acad_provenance  object
 14  acad_ecrit       object
 15  moy_finale       object
 16  mention          string
 17  abs              object
 18  exclusion        object
 19  annee            int64 
dtypes: int64(1), object(15), string(4)
memory usage: 341.3+ MB


In [ ]:
# prompt: je veux dans la colonne moy_finale tu suprime les - et et change les , pars des . et tu concervite le type en float

# Convert 'moy_finale' to string type
all_data['moy_finale'] = all_data['moy_finale'].astype(str)

# Replace '-' with empty string and ',' with '.'
all_data['moy_finale'] = all_data['moy_finale'].str.replace('-', '', regex=False)
all_data['moy_finale'] = all_data['moy_finale'].str.replace(',', '.', regex=False)

# Convert to numeric, coercing errors to NaN
all_data['moy_finale'] = pd.to_numeric(all_data['moy_finale'], errors='coerce')

# Now you can check the info
all_data.info()


<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 2236489 entries, 0 to 2236488
Data columns (total 20 columns):
 #   Column           Dtype  
---  ------           -----  
 0   prenom           object 
 1   nom              object 
 2   date_nais        string 
 3   nationalite      object 
 4   serie            string 
 5   sexe             object 
 6   age              object 
 7   etablissement    object 
 8   centre_examen    object 
 9   m_o_1            object 
 10  centre_ecrit     object 
 11  type_candidat    object 
 12  resultat         string 
 13  acad_provenance  object 
 14  acad_ecrit       object 
 15  moy_finale       float64
 16  mention          string 
 17  abs              object 
 18  exclusion        object 
 19  annee            int64  
dtypes: float64(1), int64(1), object(14), string(4)
memory usage: 341.3+ MB


In [ ]:
# prompt: affiche moi les modaliter dans resultat et le nombre

# Affiche les modalités de la colonne 'resultat' et leur nombre
resultat_counts = all_data['resultat'].value_counts()
resultat_counts


the groupby keys will be sorted anyway, although the 'sort=False' was passed. See the following issue for more details: https://github.com/modin-project/modin/issues/3571.


resultat
000         1195586
101          508803
111          379713
100          148130
10              280
Resultat          1
Name: count, dtype: int64

In [ ]:
# prompt: affiche moi la ligne avec resultat= Resultat

import pandas as pd

# Assuming 'all_data' DataFrame is already created as in your provided code

# Find rows where the 'resultat' column is equal to 'Resultat' (case-sensitive)
result_rows = all_data[all_data['resultat'] == 'Resultat']

# Print those rows
result_rows


,prenom,nom,date_nais,nationalite,serie,sexe,age,etablissement,centre_examen,m_o_1,centre_ecrit,type_candidat,resultat,acad_provenance,acad_ecrit,moy_finale,mention,abs,exclusion,annee
1708151,Prénom(s),Nom,Date Nais.,Nationalité,Série,Se-\nxe,Age,Etablissement,Centre d'examen,M.O-1,Centre d'Ecrit,Type\nCandidat,Resultat,Acad. / Ets.\nProvenance,Acad. C.\nEcrit,NaN,Mention,Abs.,Exclu-\nsion,2021


In [ ]:

all_data = all_data[all_data['resultat'] != 'Resultat']

In [ ]:
# Affiche les modalités de la colonne 'resultat' et leur nombre
resultat_counts = all_data['resultat'].value_counts()
resultat_counts

resultat
000    1195586
101     508803
111     379713
100     148130
10         280
Name: count, dtype: int64

In [ ]:
# prompt: affiche le 5 premier line ou la colonne année est 2024

print(data['2024'].head())


            prenom    nom   date_nais nationalite serie sexe  age  \
0         Aissatou  BARRY  08/06/2005     SENEGAL   L'1    F   19   
1  Safiatou Maliel     BA  15/02/2002     SENEGAL   L'1    F   22   
2  Abdoulaye Seydi  BALDE  16/10/2004     SENEGAL   L'1    M   19   
3       Adama Hawa  BALDE  04/07/2002     SENEGAL   L'1    F   21   
4         Aïssatou  BALDE  25/05/1989     SENEGAL   L'1    F   35   

                                       etablissement centre_examen    m_o_1  \
0                                  LYCEE DE MAMPATIM          DABO  Anglais   
1  CENTRE ACADEMIQUE D'ORIENTATION SCOLAIRE ET PR...          DABO  Anglais   
2                                      LYCEE DE DABO          DABO  Anglais   
3                                  LYCEE DE MAMPATIM          DABO  Anglais   
4  CENTRE ACADEMIQUE D'ORIENTATION SCOLAIRE ET PR...          DABO  Anglais   

  centre_ecrit type_candidat resultat acad_provenance acad_ecrit moy_finale  \
0  CEM DE DABO      Officiel   

In [ ]:
all_data.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 2232512 entries, 0 to 2236488
Data columns (total 20 columns):
 #   Column           Dtype  
---  ------           -----  
 0   prenom           object 
 1   nom              object 
 2   date_nais        string 
 3   nationalite      object 
 4   serie            string 
 5   sexe             object 
 6   age              object 
 7   etablissement    object 
 8   centre_examen    object 
 9   m_o_1            object 
 10  centre_ecrit     object 
 11  type_candidat    object 
 12  resultat         string 
 13  acad_provenance  object 
 14  acad_ecrit       object 
 15  moy_finale       float64
 16  mention          string 
 17  abs              object 
 18  exclusion        object 
 19  annee            int64  
dtypes: float64(1), int64(1), object(14), string(4)
memory usage: 357.7+ MB


In [ ]:
# prompt: telcharge moi cette datafram en fichier csv encodage utf-8 dans ce dossier /content/drive/MyDrive/Memoire_Bac/

# Save the DataFrame to a CSV file with UTF-8 encoding
output_file_path = "/content/drive/MyDrive/Memoire_Bac/all_data.csv"
all_data.to_csv(output_file_path,sep=";",index=False, decimal=",")

print(f"DataFrame saved to {output_file_path}")


DataFrame saved to /content/drive/MyDrive/Memoire_Bac/all_data.csv
